In [3]:
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\edge\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [7]:
!pip install rank_bm25

You should consider upgrading via the 'c:\users\edge\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


In [41]:
from nltk.tokenize import word_tokenize
import pickle, json
import pandas as pd

from rank_bm25 import *
from gensim.parsing.preprocessing import STOPWORDS
from gensim.parsing.preprocessing import remove_stopwords
import re

In [42]:
def stopwords_remove(lst):
    lst1=list()
    for str in lst:
        text_tokens = word_tokenize(str)
        tokens_without_sw = [word for word in text_tokens if not word in all_stopwords_gensim]
        str_t = " ".join(tokens_without_sw)
        lst1.append(str_t)
 
    return lst1

In [105]:
def spl_chars_removal(lst):
    lst1=list()
    for element in lst:
        str= ""
        str = re.sub("[^A-Za-z0-9]+"," ",element)
        lst1.append(str)
    return lst1

In [106]:
def remove_non_ascii(sentence):
    temp_str = sentence.encode('ascii', errors='ignore').decode("utf-8")
    result = " ".join(temp_str.split())
    return result

In [ ]:
import nltk
from nltk.stem import PorterStemmer
ps = PorterStemmer() 
sentence = "Charles IV, King of Bohemia and Holy Roman Emperor, had a long and successful reign. The Empire he ruled from Prague expanded, and his subjects lived in peace and prosperity."
for word in sentence.split():
    print(ps.stem(word))
    
def sentence_stemmer(sentence):
    return [ps.stem(word) for word in sentence]


In [ ]:
with open('master_list_relabeled.json', 'r') as fp:
    master_list = json.load(fp)
    
job_desc_docs = []
for i in master_list:
    job_desc_docs.append(master_list[i]['job_description'])

print(len(job_desc_docs))
job_desc_docs_raw = job_desc_docs

job_desc_docs = [remove_stopwords(i).split() for i in job_desc_docs]
job_desc_docs = [spl_chars_removal(i) for i in job_desc_docs]
job_desc_docs_stemmed = [sentence_stemmer(i) for i in job_desc_docs]
print(job_desc_docs_stemmed[1])

with open('job_desc_tokenized.pickle', 'wb') as handle:
    pickle.dump(job_desc_docs_stemmed, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print("saved")

In [155]:
import pandas as pd
resume_raw = pd.read_csv('Updated_dataset.csv')
resume_list = []

for resume in resume_raw['Resume']:
    resume_list.append(resume)
    
print(len(resume_list))
resume_list_raw = resume_list

resume_list = [remove_non_ascii(i) for i in resume_list]
resume_list = [remove_stopwords(i).split() for i in resume_list]
resume_list = [spl_chars_removal(i) for i in resume_list]
resume_list_stemmed = [sentence_stemmer(i) for i in resume_list]
    
print(resume_list_stemmed[0])
with open('resume_list_stemmed.pickle', 'wb') as handle:
    pickle.dump(resume_list_stemmed, handle, protocol=pickle.HIGHEST_PROTOCOL)


166
['skill', ' ', 'program', 'languages ', 'python', ' pandas ', 'numpy ', 'scipy ', 'scikit learn ', 'matplotlib ', 'sql ', 'java ', 'javascript jquery ', ' ', 'machin', 'learning ', 'regression ', 'svm ', 'na v', 'bayes ', 'knn ', 'random', 'forest ', 'decis', 'trees ', 'boost', 'techniques ', 'cluster', 'analysis ', 'word', 'embedding ', 'sentiment', 'analysis ', 'natur', 'languag', 'processing ', 'dimension', 'reduction ', 'topic', 'model', ' lda ', 'nmf ', 'pca', ' ', 'neural', 'nets ', ' ', 'databas', 'visualizations ', 'mysql ', 'sqlserver ', 'cassandra ', 'hbase ', 'elasticsearch', 'd3 js ', 'dc js ', 'plotly ', 'kibana ', 'matplotlib ', 'ggplot ', 'tableau ', ' ', 'others ', 'regular', 'expression ', 'html ', 'css ', 'angular', '6 ', 'logstash ', 'kafka ', 'python', 'flask ', 'git ', 'docker ', 'vision', ' ', 'open', 'cv', 'understand', 'deep', 'learning educ', 'detail', 'data', 'scienc', 'assur', 'associ', 'data', 'scienc', 'assur', 'associ', ' ', 'ernst', ' ', 'young', 'llp

In [159]:
tokenized_corpus = job_desc_docs_stemmed
bm25 = BM25Okapi(tokenized_corpus)

df = pd.DataFrame(columns = ["resume", "job"])

# for idx in range(250,280):
for idx in range(len(resume_list_stemmed)):
    tokenized_query = resume_list_stemmed[idx]
# print("---------------------Resume-----------------------")
# print(resume_list_raw[idx])
# print("---------------------JD-----------------------")

#     doc_top = bm25.get_top_n(tokenized_query, job_desc_docs_raw, n=5)
    # print(doc_top[0])
    doc_scores = bm25.get_scores(tokenized_query)
#     doc_scores_dict = {key: value for (key, value) in enumerate(doc_scores)}
    top_matches = sorted(range(len(doc_scores)), key=lambda i: doc_scores[i])[-8:]
    print(top_matches)
    
    counter = 0
#     for doc_idx, score in enumerate(doc_scores):
#         if score == max(doc_scores):
#             print(idx, doc_idx)
    for match in top_matches:
        # print("match")
        df = df.append({"resume": remove_non_ascii(resume_list_raw[idx]), "job": job_desc_docs_raw[match]}, ignore_index=True)
            

df.to_csv('matches.tsv', sep = '\t')            

[4296, 5580, 6199, 4482, 4236, 6193, 3238, 2400]
[7107, 2868, 6193, 1991, 5592, 6546, 2701, 2135]
[6546, 3235, 2959, 5592, 6176, 4613, 2135, 2400]
[4803, 6193, 2898, 338, 4482, 4173, 4260, 4236]
[2434, 4297, 2033, 765, 2701, 1991, 6546, 2135]
[2868, 4412, 5937, 6546, 5580, 2701, 1991, 2135]
[4648, 2344, 4296, 6199, 4236, 3238, 4482, 6193]
[6243, 6685, 5580, 5872, 6768, 2400, 5560, 6193]
[6763, 5713, 7212, 5912, 6243, 6193, 6768, 5078]
[6243, 5580, 6366, 3238, 6193, 4236, 4271, 4482]
[296, 2290, 3685, 5553, 3631, 613, 6091, 5057]
[679, 2693, 5403, 25, 7042, 5119, 4808, 3602]
[184, 1916, 2154, 2343, 3166, 7240, 5119, 5697]
[2179, 3028, 2921, 1482, 5119, 3427, 3166, 3234]
[3645, 279, 1266, 3383, 2631, 5697, 4062, 3166]
[2179, 3049, 1266, 3028, 184, 5119, 1482, 3166]
[6783, 51, 5693, 552, 1786, 7380, 2631, 3645]
[5534, 3166, 1987, 5794, 1463, 1164, 20, 6853]
[5753, 1656, 3685, 4063, 1401, 2631, 1266, 3645]
[2929, 1786, 6259, 7380, 3605, 2631, 7276, 3645]
[6526, 2516, 6760, 5033, 2339, 6914

In [ ]:
print(master_list[str(6259)])